In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pycbc import psd as psd_func
from simple_pe.waveforms import make_waveform

# Disable pesummary warnings
import logging
_logger = logging.getLogger('PESummary')
_logger.setLevel(logging.CRITICAL + 10)

# Otherwise plt.savefig crashes with latex errors
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

# Plotting harmonics

In [ ]:
f_low = 20
sample_rate = 4096
tlen = 32

# Create psd
ifos = ['H1']
psds = {'H1': 'aLIGOZeroDetHighPower',
        'f_low': f_low,
        'f_high': int(sample_rate/2),
        'length': tlen,
        'delta_f': 1. / tlen
        }
psd = {}
for ifo in ifos:
    psd[ifo] = psd_func.analytical.from_string(psds[ifo], psds['length'] * psds['f_high'] + 1, psds['delta_f'],
                                                     psds['f_low'])

params = {'ecc10sqrd': 0.2**2, 'chirp_mass': 24, 'symmetric_mass_ratio': 2/9, 'chi_eff': 0, 'distance': 1}
hps = make_waveform(params, psd['H1'].delta_f, f_low, len(psd['H1']), approximant='TEOBResumS-Dali-Harms', n_ecc_harms=4)['H1']
hps = {key: hps[key].to_timeseries() for key in hps.keys()}

In [ ]:
for key in hps.keys():
    plt.figure(figsize=(6.4*2.5, 4.8))
    plt.plot(hps[key].sample_times, hps[key], label=str(key))
    plt.legend()
    plt.xlim(-3, 0.1)
    plt.show()

# Frequency as function of time

In [ ]:
hp, hc = make_waveform(params, psd['H1'].delta_f, f_low, len(psd['H1']), approximant='TEOBResumS-Dali', return_hc=True)
h = hp.to_timeseries() - 1j*hc.to_timeseries()

In [ ]:
plt.plot(h.sample_times, h)
plt.xlim(-1, 0.1)
plt.show()

In [ ]:
ang_diff = np.diff(np.angle(h[10:3500]))
ang_diff = 2*np.pi - ang_diff % (2*np.pi)
plt.hist(ang_diff, bins=50, histtype='step')
plt.show()

In [ ]:
plt.plot(h.sample_times, h)
plt.xlim(-1, 0.1)
plt.show()

In [ ]:
plt.hist(ang_diff)
plt.show()

In [ ]:
ang_diff = np.diff(np.angle(hps[0]))
ang_diff = 2*np.pi - ang_diff % (2*np.pi)
f_inst = ang_diff*sample_rate/(2*np.pi)
plt.plot(hps[0].sample_times[:-1], f_inst)
plt.xlim(-1, 0.1)
# plt.ylim(0, 200)
plt.show()